## Introduction.
<p>RMS Titanic was a British passenger liner that sank in the North Atlantic Ocean in the early hours of 15 April 1912, after colliding with an iceberg during its maiden voyage from Southampton to New York City. It is one of the most infamous shipwrecks in history. Alot of people lot their lives.</p>
<p>One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
</p>
<p>We seek to analyse the data to findout which groups were likely to survive.</p>

<p>Inorder to analyse and do some machine learning we require to import a couple of libraries.</p>

In [3]:
# data analysis 
import numpy as np
import pandas as pd
import random as rnd

# ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
